In [3]:
import mesa
from mesa.experimental import JupyterViz

import seaborn as sns
import numpy as np
import pandas as pd

# Agent-Based Modeling with Mesa: Tutorial Notebook

This notebook provides a step-by-step guide to creating an agent-based model using the Mesa library. 
As we progress through the tutorial, we'll cover various aspects of creating, running, and analyzing agent-based models.

**Table of Contents:**
1. [Creating agents](#Creating-agents)
2. [Creating the Model](#Creating-the-Model)
3. [Adding the Scheduler](#Adding-the-Scheduler)
4. [Agent Step](#Agent-Step)
5. [Running the Model](#Running-the-Model)

## Creating agents

Agents are the individual entities that act in the model. They are essential components of agent-based models, representing individual entities that interact within the system. In our model, agents represent individuals exchanging money, each with a unique identifier and wealth. In this section, we'll create the MoneyAgent class, which is an agent for part of a basic economic model. To create the MoneyAgent class, we extend the `mesa.Agent` class. This class contains attributes such as wealth and a unique identifier.

In [4]:
from mesa import Agent

class MoneyAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1  # Initial wealth for each agent

## Creating the Model

The model represents the overall environment in which agents interact. It holds and manages all the agents on the grid and evolves in discrete time steps. In this section, we'll create the MoneyModel class, which manages agents and the model's evolution over time. To make the MoneyModel class, we extend the `mesa.Model` class. The model initializes with a specified number of agents and creates a scheduler to control agent activation.

In [5]:
from mesa import Model

class MoneyModel(Model):
    def __init__(self, num_agents):
        super().__init__()
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)
        
        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)

## Adding the Scheduler

The scheduler dictates agent activation order as well as model progression by one step; A step (also called tick) is the smallest unit of time in the model. In the example, the model will use `RandomActivation`, which activates all agents in a random order each step. Other Schdeulers types are `BaseScheduler` `RandomActivation` `SimultaneousActivation` `StagedActivation` `RandomActivationByType` `DiscreteEventScheduler`

In [6]:
class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        # Pass the parameters to the parent class.
        super().__init__(unique_id, model)

        # Create the agent's attribute and set the initial values.
        self.wealth = 1

    def step(self):
        # The agent's step will go here.
        # For demonstration purposes we will print the agent's unique_id
        print(f"Hi, I am an agent, you can call me {str(self.unique_id)}.")


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N):
        super().__init__()
        self.num_agents = N
        # Create scheduler and assign it to the model
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            # Add the agent to the scheduler
            self.schedule.add(a)

    def step(self):
        """Advance the model by one step."""

        # The model's step will go here for now this will call the step method of each agent and print the agent's unique_id
        self.schedule.step()

## Running the Model

In this section, we'll run a basic simulation of our model by creating a model object and calling the `step` method.

In [9]:
num_agents = 10
model = MoneyModel(num_agents)
model.step()

Hi, I am an agent, you can call me 1.
Hi, I am an agent, you can call me 2.
Hi, I am an agent, you can call me 5.
Hi, I am an agent, you can call me 4.
Hi, I am an agent, you can call me 6.
Hi, I am an agent, you can call me 0.
Hi, I am an agent, you can call me 8.
Hi, I am an agent, you can call me 9.
Hi, I am an agent, you can call me 3.
Hi, I am an agent, you can call me 7.


### exercise
 - Change the output of the agent 
 - run the model a couple of times

## Agent Step

This section outlines the behavior of agents within each step of the model. Agents evaluate their wealth at each step and, if they possess funds, transfer one unit to a randomly selected agent. The agent's step method is invoked by the scheduler at every model step. To enable random agent selection, the model's random-number generator is utilized.

In [10]:
%%writefile starter_model/money_model.py
import mesa

# Defining agent step method
class MoneyAgent(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1  # Initial wealth for each agent

    def step(self):
        if self.wealth > 0:
            other_agent = self.random.choice(self.model.schedule.agents)
            other_agent.wealth += 1
            self.wealth -= 1


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N):
        super().__init__()
        self.num_agents = N
        # Create scheduler and assign it to the model
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            # Add the agent to the scheduler
            self.schedule.add(a)

    def step(self):
        """Advance the model by one step."""

        # The model's step will go here for now this will call the step method of each agent and print the agent's unique_id
        self.schedule.step()

Overwriting starter_model/money_model.py


In [11]:
%%writefile starter_model/run.py
from money_model import MoneyModel

model = MoneyModel(10)
for i in range(10):
    model.step()

Overwriting starter_model/run.py


In [12]:
!python -W ignore starter_model/run.py

### exercise
- delete -W ignore from last cell
 - change the code so the it chooses $k<n$ ($n$ number of agent in the simulation) agents instead of 1

In [13]:
%%writefile starter_model/run.py
from money_model import MoneyModel
import seaborn as sns

all_wealth = []
# This runs the model 100 times, each model executing 10 steps.
for j in range(1000):
    # Run the model
    model = MoneyModel(10)
    for i in range(10):
        model.step()

    # Store the results
    for agent in model.schedule.agents:
        all_wealth.append(agent.wealth)

# Use seaborn
g = sns.histplot(all_wealth, discrete=True, stat='density')
g.set(title="Wealth distribution", xlabel="Wealth", ylabel="Number of agents");
g.figure.savefig("output.png")

Overwriting starter_model/run.py


In [14]:
!python -W ignore starter_model/run.py